In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install fairseq==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 10.8 MB/s 
     |████████████████████████████████| 118 kB 70.1 MB/s 
     |████████████████████████████████| 235 kB 71.6 MB/s 
     |████████████████████████████████| 123 kB 68.1 MB/s 
     |████████████████████████████████| 112 kB 68.3 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141231 sha256=88e4a31bf1f8c73ac1d0a7a3b4fbf141c8507c9b85d84aea81c5209ad90fdf2d
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [ ]:
!wget "https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/pretrained_models/ja-en/small.tar.gz"

!tar -zxvf small.tar.gz
# https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/
# https://www.otofu.org/papers/morishita20nlp.pdf

--2022-11-30 12:25:25--  https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/pretrained_models/ja-en/small.tar.gz
Resolving www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)... 163.137.218.58
Connecting to www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)|163.137.218.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 814487726 (777M) [application/x-gzip]
Saving to: ‘small.tar.gz’

small.tar.gz        100%[===================>] 776.76M   569KB/s    in 23m 47s 

2022-11-30 12:49:14 (557 KB/s) - ‘small.tar.gz’ saved [814487726/814487726]

small/
small/dict.en.txt
small/LICENSE
small/small.pretrain.pt
small/dict.ja.txt


#sentencepiece

In [ ]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 13.1 MB/s 


In [ ]:
!wget https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/spm_models/en-ja_spm.tar.gz
!tar -zxvf en-ja_spm.tar.gz

--2022-11-30 12:49:28--  https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/spm_models/en-ja_spm.tar.gz
Resolving www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)... 163.137.218.58
Connecting to www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)|163.137.218.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1138921 (1.1M) [application/x-gzip]
Saving to: ‘en-ja_spm.tar.gz’

en-ja_spm.tar.gz    100%[===================>]   1.09M   447KB/s    in 2.5s    

2022-11-30 12:49:32 (447 KB/s) - ‘en-ja_spm.tar.gz’ saved [1138921/1138921]

enja_spm_models/
enja_spm_models/spm.en.nopretok.vocab
enja_spm_models/spm.en.nopretok.model
enja_spm_models/spm.ja.nopretok.model
enja_spm_models/spm.ja.nopretok.vocab


In [ ]:
# 同封のsentencepieceをロード https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/
import sentencepiece as spm
sp_ja = spm.SentencePieceProcessor()
sp_en = spm.SentencePieceProcessor()
sp_ja.load('/content/enja_spm_models/spm.ja.nopretok.model')
sp_en.load('/content/enja_spm_models/spm.en.nopretok.model')

True

In [ ]:
#import zenhan
def sentencepiece_en(outfile, infile):
    fout = open(outfile, "w")
    fin = open(infile, "r")
    for line in fin:
        text = " ".join(sp_en.encode_as_pieces(line.strip()))
        fout.write(text + "\n")
    fin.close()
    fout.close()

def sentencepiece_ja(outfile, infile):
    fout = open(outfile, "w")
    fin = open(infile, "r")
    for line in fin:
        text = " ".join(sp_ja.encode_as_pieces(line.strip()))
        fout.write(text + "\n")
    fin.close()
    fout.close()

In [ ]:
# 前処理（日本語の単語分割）sentencepiece

# train
sentencepiece_ja("train.ja", "/content/drive/MyDrive/exp5-4-1/ja.comp.train.txt")
sentencepiece_en("train.en", "/content/drive/MyDrive/exp5-4-1/en.train.txt")

# valid
sentencepiece_ja("valid.ja", "/content/drive/MyDrive/exp5-4-1/ja.comp.valid.txt")
sentencepiece_en("valid.en", "/content/drive/MyDrive/exp5-4-1/en.valid.txt")

# test
sentencepiece_ja("test.ja", "/content/drive/MyDrive/exp5-4-1/ja.comp.test.txt")
sentencepiece_en("test.en", "/content/drive/MyDrive/exp5-4-1/en.test.txt")

# eval
sentencepiece_ja("eval.ja", "/content/drive/MyDrive/exp5-4-1/ja.comp.eval.txt")

#共通の処理

In [ ]:
from torchtext.data.metrics import bleu_score

def bleu(fname_ref, fname_pred):    
    # 正解文
    fin = open(fname_ref, "r")
    refs = [[line.strip().split()] for line in fin]
    fin.close()
    # 出力文
    fin = open(fname_pred, "r")
    preds = [line.strip().split() for line in fin]
    fin.close()
    # BLEU
    score = bleu_score(preds, refs)
    print("BLEU = %.2f" % (score*100))

In [ ]:
# データセットの準備

! fairseq-preprocess --source-lang ja --target-lang en --trainpref train --validpref valid --testpref test --tgtdict small/dict.en.txt --srcdict small/dict.ja.txt

2022-11-30 15:01:28 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-11-30 15:01:28 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False

In [ ]:
# 訓練
! fairseq-train data-bin --arch transformer --restore-file /content/small/small.pretrain.pt \
    --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --encoder-ffn-embed-dim 1024 --decoder-ffn-embed-dim 1024 \
    --encoder-attention-heads 4 --decoder-attention-heads 4 \
    --batch-size 128 --source-lang ja --target-lang en \
    --optimizer adam --lr-scheduler inverse_sqrt --lr 1e-4 --warmup-updates 400 \
    --dropout 0.25 --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
    --no-epoch-checkpoints --seed 5 --patience 3 

2022-11-30 17:13:05 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-11-30 17:13:07 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 5, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': Fal

In [ ]:
# テストデータに対する翻訳

! fairseq-interactive data-bin --path /content/checkpoints/checkpoint_best.pt --batch-size 256 --buffer-size 512 < test.ja \
--gen-subset test | grep "^H-" | sort -V | cut -f3 > fairseq.en.test.txt
ref_file = "/content/test.en"
bleu(ref_file, "fairseq.en.test.txt")

2022-11-30 16:54:11 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-11-30 16:54:12 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging

In [ ]:
# 提出データに対する翻訳
! fairseq-interactive data-bin --path /content/checkpoints/checkpoint_best.pt --batch-size 256 --buffer-size 512 < eval.ja \
--gen-subset test | grep "^H-" | sort -V | cut -f3 > fairseq.en.eval.txt

2022-11-30 17:23:56 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-11-30 17:23:58 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging

#提出用ファイルをデコード

In [ ]:
# 単語分割器 mosestokenizer

! pip install mosestokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49188 sha256=117664fd019b920b76031440258270ec82c8c5ea54f146dc773212ffb9ecedf9
  Stored in directory: /root/.cache/pip/wheels/b0/35/f7/af1258779a0b890abc3c79481460c597cb1f3659d0603cfb9d
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=5e7d5e1d9c99bf13798d5d896268d4d428334b00505b8fd3b59576a8d112ff04
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3353 sha256=7ddf6c0057dead9ea353467f0d5e958aae0447152372b03ca51e5d63fb51c18f
  Stored in directory: /root/.cache/pip/wheels/c5/4f/33/54741ffe08e38ececb1d28068a153729b4fe820bafa0a0691f
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6162 sha256=053e10af1328b14

In [ ]:
# 機械翻訳

from mosestokenizer import MosesTokenizer
tokenizer = MosesTokenizer("en", "no-escape")

fname =  "/content/fairseq.en.eval.txt"
fout = open(fname.replace(".txt", ".tok.txt"), "w")
fin = open(fname, "r")
for line in fin:
    text = sp_en.decode_pieces(line.split()).strip('_')
    fout.write(" ".join([word for word in tokenizer(text.strip().lower())]) + "\n")
fin.close()
fout.close()